In [ ]:
# Dependencies & Setup to work with BLS
# Note there is a limit to how many free pulls may be done in a day from BLS: https://www.bls.gov/developers/api_FAQs.htm#register1

import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import config
headers = {'Content-type': 'application/json'}

base_url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'

# BLS Series IDs are built on a prefix specific to the data series available + location + suffix structure
# BLS City codes: 
city_codes = {"Austin":"CT4805000000000", "Chicago":"CT1714000000000", "Denver":"CT0820000000000", "Las Vegas":"CT3240000000000",
              "Memphis":"CT4748000000000", "New York":"CT3651000000000", "Pittsburgh":"CT4261000000000", "San Francisco":"CT0667000000000"}
# BLS State codes + 00000 for "all areas":
state_codes = {"Texas":"4800000", "Illinois":"1700000", "Colorado":"0800000", "Nevada":"3200000", 
               "Tennessee":"4700000", "New York":"3600000", "Pennsylvania":"4200000", "California":"0600000"}
# prefix codes unadjsted for seasonal periods
prefix_codes = {"Unemployment":"LAU", "Non-Farm Job Openings":"JTU000000"}
# depending on the prefix, the suffix are different lengths and mean different things
suffix_codes = {"Unemployment Rate":"03", "All Size Job Openings Rate":"00R"}

# Create list of Series IDs for unemployment rate
unemp_series = []
for city in city_codes:
    unemp_series.append(prefix_codes["Unemployment"] + city_codes[city] + suffix_codes["Unemployment Rate"])

# Create list of Series IDs for job openings rate (only at state level)
jobopen_series = []
for state in state_codes:
    jobopen_series.append(prefix_codes["Non-Farm Job Openings"] + state_codes[state] + suffix_codes["All Size Job Openings Rate"])
    
# BLS queries all have a start and end year input
startyear = 2022
endyear = 2022

In [ ]:
# Request Unemployment Rate (a monthly output by year by city, county, state, area depending on your ID)

data = json.dumps({"seriesid":unemp_series,"startyear":startyear, "endyear":endyear})
p = requests.post(base_url, data=data, headers=headers)
unemp_data = json.loads(p.text)

# Print response characteristics to identify potential issues such as maximizing allowed queries in a day
print(p)
print(f"Request Status {unemp_data['status']}")
print(f"Request Message {unemp_data['message']}")
      
# Reduce to just the results series of data
unemp_data_core = unemp_data["Results"]["series"]

In [ ]:
# Create a DataFrame for Unemployment Rates Over Time

# Create empty lists to hold data collected from json output
city_list = []
time_headers = []
rate_list = []

# Iterate through city data (json data is setup as a list of data by SeriesID (by city for the unemp_data_core))
for i in range(len (unemp_data_core)):

    # trim the series ID to pull out the city code and programmatically populate city rather than relying on original input staying in same order
    city_value = unemp_data_core[i]["seriesID"][3:-2]
    # Reference for pulling dictionary key from known value: https://www.geeksforgeeks.org/python-get-key-from-value-in-dictionary/
    city_key = list(filter(lambda c: city_codes[c] == city_value, city_codes))[0]
    city_list.append(city_key)
    # Setup list for time period and rate to be used for single city
    th = []
    rl = []
    # Pull out the time period and rate value storing them in the single sity list
    for j in range(len(unemp_data_core[i]["data"])):
        th.append(unemp_data_core[i]["data"][j]["year"]+" "+unemp_data_core[i]["data"][j]["period"])
        rl.append(unemp_data_core[i]["data"][j]["value"])

    # Convert the unemployment rates from string to number
    rl_num = list(map(float, rl))
    
    # Reverse lists so oldest to newest
    th_rev = th[::-1]
    rl_num_rev = rl_num[::-1]
    
    # Add the single city list into the overall list
    time_headers.append(th_rev) # This should be the same for each city based on common output
    rate_list.append(rl_num_rev) # This will be different for each city

# iterate through the city and rate lists as columns to use pandas plot with lines per city
unemp_dict = {}
for i in range(len(city_list)):
    unemp_dict[city_list[i]] = rate_list[i]
    
unemp_df = pd.DataFrame(unemp_dict, index=time_headers[0])
unemp_df

In [ ]:
# Create plot with legend to the side of the plot
unemp_plot = unemp_df.plot.line(figsize=(10, 5), xlabel="Month", ylabel="Unemployment Rate (%)", title="Unemployment Rate Over Time")
plt.legend(bbox_to_anchor=(1.05,1.0), loc="upper left")
plt.show()

In [ ]:
# Request Job Opening Rate (a monthly output by year by city, county, state, area depending on your ID)

data = json.dumps({"seriesid":jobopen_series,"startyear":startyear, "endyear":endyear})
p = requests.post(base_url, data=data, headers=headers)
jobopen_data = json.loads(p.text)

# Print response characteristics to identify potential issues such as maximizing allowed queries in a day
print(p)
print(f"Request Status {jobopen_data["status"]}")
print(f"Request Message {jobopen_data["message"]}")

# Reduce to just the results series of data
jobopen_data_core = jobopen_data["Results"]["series"]

In [ ]:
# Create a DataFrame for Job Opening Rates Over Time

# Create empty lists to hold data collected from json output
state_list = []
time_periods = []
rate_list = []

# Iterate through city data (json data is setup as a list of data by SeriesID (by city for the unemp_data_core))
for i in range(len(jobopen_data_core)):

    # trim the series ID to pull out the city code and programmatically populate city rather than relying on original input staying in same order 4800000
    state_value = jobopen_data_core[i]["seriesID"][9:-5]

    # Reference for pulling dictionary key from known value: https://www.geeksforgeeks.org/python-get-key-from-value-in-dictionary/
    state_key = list(filter(lambda s: state_codes[s] == state_value, state_codes))[0]
    state_list.append(state_key)

    # Setup list for time period and rate to be used for single city
    th = []
    rl = []
    # Pull out the time period and rate value storing them in the single sity list
    for j in range(len(jobopen_data_core[i]["data"])):
        th.append(jobopen_data_core[i]["data"][j]["year"]+" "+jobopen_data_core[i]["data"][j]["period"])
        rl.append(jobopen_data_core[i]["data"][j]["value"])

    # Convert the unemployment rates from string to number
    rl_num = list(map(float, rl))
    
    # Reverse lists so oldest to newest
    th_rev = th[::-1]
    rl_num_rev = rl_num[::-1]
    
    # Add the single city list into the overall list
    time_periods.append(th_rev) # This should be the same for each city based on common output
    rate_list.append(rl_num_rev) # This will be different for each city

# iterate through the city and rate lists to add the city as the index and the rates in the columns
jobopen_dict = {}
for i in range(len(state_list)):
    jobopen_dict[state_list[i]] = rate_list[i]

jobopen_df = pd.DataFrame(jobopen_dict, index=time_periods[0])
jobopen_df

In [ ]:
# Create plot with legend to the side of the plot
jobopen_plot = jobopen_df.plot.line(figsize=(10, 5), xlabel="Month", ylabel="Job Openings Rate (%)", title="Non-Farm Job Openings Rate Over Time")
plt.legend(bbox_to_anchor=(1.05,1.0), loc="upper left")
plt.show()